In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import keras
from keras.preprocessing.text import Tokenizer
import sklearn
import sklearn.preprocessing
import tensorflow as tf

Using TensorFlow backend.


In [5]:
import pandas as pd

In [31]:

def clean_chinese_str(s):
    out = ''
    #mystr = string.decode('utf-8')
    mystr = s
    mystr = mystr[:100]
    for letter in mystr:
        out += letter + ' '
    return out

def preprocess_text(texts):
    import jieba
    return texts.map(lambda x:' '.join(list(jieba.cut(x))))

# 加载数据,对文本进行预处理。返回texts, labels.
def load_csv_data(path):
    df_train = pd.read_csv(path)
    data_list = df_train['text']
    class_list = df_train['label']
    y_labels = list(class_list.value_counts().index)
    label_encoder = sklearn.preprocessing.LabelEncoder()
    label_encoder.fit(y_labels)
    num_labels = len(y_labels)
    y_labels = keras.utils.to_categorical(class_list.map(lambda x: label_encoder.transform([x])), num_labels)
    texts = preprocess_text(data_list)
    return texts, y_labels

def tokenize_texts(texts):
    tokenizer = Tokenizer(filters='!"#$%&()*,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=" ")
    tokenizer.fit_on_texts(texts)
    vocab = tokenizer.word_index
    word_ids = tokenizer.texts_to_sequences(texts)
    padded_seqs = keras.preprocessing.sequence.pad_sequences(
        word_ids, padding='post', truncating='post', maxlen=100)
    return padded_seqs, vocab

def split_train_test(X, y):
    train_count = len(X)
    X_train = X[0:train_count]
    Y_train = y[0:train_count]
    X_test = X[train_count:]
    Y_test = y[train_count:]
    return X_train, Y_train, X_test, Y_test

def load_embedding_layer(word_index, input_length, embedding_path):
    import yznlp.wordvec
    
    embeddings, dim = yznlp.wordvec.load_tencent_word_embedding(embedding_path)
    weights = np.zeros((len(word_index), dim ))
    unknown_words = []
    for word, i in word_index.items():
        
        vec = embeddings.get(word, None)
        print(word, i, vec)
        if vec is not None:
            weights[i-1] = vec
        else:
            unknown_words.append((word, i))
    print(len(unknown_words))
    return keras.layers.Embedding(len(word_index), dim, input_length=input_length, trainable=False, weights=[weights])

def get_tencent_lstm_model(word_index, input_length, embedding_path):
    embedding_layer = load_embedding_layer(word_index, input_length, embedding_path)
    model3 = keras.models.Sequential()
    model3.add(embedding_layer)
    model3.add(keras.layers.LSTM(256, dropout=0.5, recurrent_dropout=0.1))
    model3.add(keras.layers.Dense(256, activation='relu'))
    model3.add(keras.layers.Dense(2, activation='softmax'))
    model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model3

    
    
texts, labels = load_csv_data('data/ad1/ch_ad.csv')
seqs, vocab = tokenize_texts(texts)
X_train, Y_train, X_test, Y_test = split_train_test(seqs, labels)


In [ ]:
sess = tf.Session(config=tf.ConfigProto(device_count={'gpu':1},log_device_placement=True))
import keras.backend.tensorflow_backend as KTF
KTF.set_session(sess)
model3 = get_tencent_lstm_model(vocab, 100, 'var/Tencent_AILab_ChineseEmbedding.txt')
model3.fit(X_train, Y_train, batch_size=32, epochs=12, validation_data=(X_test, Y_test))

In [29]:
import numpy as np
x_train_padded_seqs=np.expand_dims(X_train,axis=2)
x_train_padded_seqs[0]

array([[ 1603],
       [   40],
       [  417],
       [14389],
       [  530],
       [  330],
       [  950],
       [  931],
       [  110],
       [ 4499],
       [ 4500],
       [ 4501],
       [14390],
       [ 2498],
       [ 1604],
       [ 5154],
       [    6],
       [ 1437],
       [   23],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [    0],
       [

In [26]:
x_test_padded_seqs=np.expand_dims(X_test,axis=2)

NameError: name 'tf' is not defined

In [31]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(256, dropout=0.5, recurrent_dropout=0.1))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(2, activation='softmax'))


In [32]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:

model.fit(x_train_padded_seqs, Y_train, batch_size=32, epochs=12, validation_data=(x_test_padded_seqs, Y_test))

Train on 14027 samples, validate on 1559 samples
Epoch 1/12
14027/14027 [==============================] - 87s 6ms/step - loss: 0.6711 - acc: 0.5573 - val_loss: 0.8392 - val_acc: 0.6594
Epoch 2/12
14027/14027 [==============================] - 86s 6ms/step - loss: 0.6440 - acc: 0.6132 - val_loss: 1.1139 - val_acc: 0.3149
Epoch 3/12
14027/14027 [==============================] - 86s 6ms/step - loss: 0.6021 - acc: 0.6759 - val_loss: 0.8329 - val_acc: 0.4920
Epoch 12/12
14027/14027 [==============================] - 86s 6ms/step - loss: 0.6110 - acc: 0.6700 - val_loss: 0.9513 - val_acc: 0.5247


In [ ]:
model2 = keras.models.Sequential()


In [ ]:
#model2.add(keras.layers.Input(shape=(100,), dtype='int32'))
model2.add(keras.layers.Embedding(len(tokenizer.word_index)+1, 300, input_length=100))
model2.add(keras.layers.LSTM(256, dropout=0.5, recurrent_dropout=0.1))
model2.add(keras.layers.Dense(256, activation='relu'))
model2.add(keras.layers.Dense(2, activation='softmax'))


In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_train = keras.preprocessing.sequence.pad_sequences(x_train_word_ids, padding='post', truncating='post', maxlen=100)
X_test = keras.preprocessing.sequence.pad_sequences(x_test_word_ids, padding='post', truncating='post', maxlen=100)

In [ ]:
model2.fit(X_train, Y_train, batch_size=32, epochs=12, validation_data=(X_test
                                                                                 , Y_test))